In [1]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import date
from itertools import cycle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle

## Getting links

In [56]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless=False
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [31]:
website_links = ['https://www.hornbach.de/shop/Wandverkleidung-Deckenverkleidung/S13928/artikelliste.html',
                'https://www.hornbach.de/shop/Modellbau/S20347/artikelliste.html',
                'https://www.hornbach.de/shop/Moebelbau/S1941/artikelliste.html',
                'https://www.hornbach.de/shop/Bauholz/S1940/artikelliste.html',
                'https://www.hornbach.de/shop/OSB-Platten-Verlegeplatten/S610/artikelliste.html',
                'https://www.hornbach.de/shop/Gartenbau-Landschaftsbau/Terrassenbau/Terrassendielen/Holz-Terrassendielen/S840/artikelliste.html',
                'https://www.hornbach.de/shop/Gartenbau-Landschaftsbau/Terrassenbau/Terrassendielen/Unterkonstruktionen/S843/artikelliste.html']

In [2122]:
all_links = []
for s, url in enumerate(website_links):
    browser.get(url)
    section_links = []
    path = '//*[@class="paging-indicator ng-binding"]'
    pages = browser.find_elements_by_xpath(path)[0].text
    pages = int(pages.split('von ')[-1])
    
    print('Section', s+1, "Pages", pages, url)
    for i in range(1): #pages-1
        sleep(2)
        if i > 0:
            path = '//*[@class="paging-btn right"]'
            browser.find_elements_by_xpath(path)[0].click()
        else:
            pass
        sleep(2)
        path = '//*[@id="article-list"]//a'
        page_links_raw = browser.find_elements_by_xpath(path)
        page_links=[]
        for url in page_links_raw:
            link = url.get_attribute('href')
            if link not in page_links and 'artikel' in link and 'ratings' not in link  and 'artikelvergleich' not in link:
                page_links.append(link)
        
        #print(i, 'Page Links', len(page_links))
        section_links.extend(page_links)
        print(i+1,'Section Links', len(section_links))
    
    all_links.extend(section_links)
    print('All Links', len(all_links))

Section 1 Pages 14 https://www.hornbach.de/shop/Wandverkleidung-Deckenverkleidung/S13928/artikelliste.html
1 Section Links 36
All Links 1536
Section 2 Pages 5 https://www.hornbach.de/shop/Modellbau/S20347/artikelliste.html
1 Section Links 36
All Links 1572
Section 3 Pages 11 https://www.hornbach.de/shop/Moebelbau/S1941/artikelliste.html
1 Section Links 36
All Links 1608
Section 4 Pages 9 https://www.hornbach.de/shop/Bauholz/S1940/artikelliste.html
1 Section Links 36
All Links 1644
Section 5 Pages 1 https://www.hornbach.de/shop/OSB-Platten-Verlegeplatten/S610/artikelliste.html
1 Section Links 13
All Links 1657
Section 6 Pages 1 https://www.hornbach.de/shop/Gartenbau-Landschaftsbau/Terrassenbau/Terrassendielen/Holz-Terrassendielen/S840/artikelliste.html
1 Section Links 33
All Links 1690
Section 7 Pages 1 https://www.hornbach.de/shop/Gartenbau-Landschaftsbau/Terrassenbau/Terrassendielen/Unterkonstruktionen/S843/artikelliste.html
1 Section Links 28
All Links 1718


In [ ]:
section_links.extend(page_links)

In [49]:
path = '//*[@id="article-list"]//a'
page_links_raw = browser.find_elements_by_xpath(path)
page_links=[]
for url in page_links_raw:
    link = url.get_attribute('href')
    if link not in page_links and 'artikel' in link and 'ratings' not in link  and 'artikelvergleich' not in link:
        page_links.append(link)
len(page_links)

28

In [51]:
all_links.extend(page_links)
len(all_links)

1245

In [2123]:
with open('HornBachLInks.pkl', 'rb') as f:
    old_urls = pickle.load(f)

In [2124]:
missed_urls = [x for x in all_links if x not in old_urls]
len(missed_urls)

133

### Testing Options

In [372]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2126]:
%%time
#options_df = pd.DataFrame(columns=['URL','Options'], index=range(len(all_links)))
missed_opts = pd.DataFrame(columns=['URL','Options'], index=range(len(missed_urls)))
for i in range(len(missed_urls)): #all_links
    sleep(1)
    url = missed_urls[i]
    browser.get(url)
    print('Link #', i)
    
    # 'Clicking Buttons'
    try:
        path = '//*[contains(@data-selenium-test, "selectionButton")]'
        button = browser.find_elements_by_xpath(path)[0].text
        if button:
            missed_opts['Options'].loc[i]='Button'
            #print('Button')
    except:
        pass
    
    # 'Toggle'
    try:
        path = '//*[contains(@class, "Component-dropdown")]'
        toggle = browser.find_elements_by_xpath(path)[0].text
        if toggle:
            missed_opts['Options'].loc[i]='Toggle'
            #print('Toggle')
    except:
        pass

Link # 0
Link # 1
Link # 2
Link # 3
Link # 4
Link # 5
Link # 6
Link # 7
Link # 8
Link # 9
Link # 10
Link # 11
Link # 12
Link # 13
Link # 14
Link # 15
Link # 16
Link # 17
Link # 18
Link # 19
Link # 20
Link # 21
Link # 22
Link # 23
Link # 24
Link # 25
Link # 26
Link # 27
Link # 28
Link # 29
Link # 30
Link # 31
Link # 32
Link # 33
Link # 34
Link # 35
Link # 36
Link # 37
Link # 38
Link # 39
Link # 40
Link # 41
Link # 42
Link # 43
Link # 44
Link # 45
Link # 46
Link # 47
Link # 48
Link # 49
Link # 50
Link # 51
Link # 52
Link # 53
Link # 54
Link # 55
Link # 56
Link # 57
Link # 58
Link # 59
Link # 60
Link # 61
Link # 62
Link # 63
Link # 64
Link # 65
Link # 66
Link # 67
Link # 68
Link # 69
Link # 70
Link # 71
Link # 72
Link # 73
Link # 74
Link # 75
Link # 76
Link # 77
Link # 78
Link # 79
Link # 80
Link # 81
Link # 82
Link # 83
Link # 84
Link # 85
Link # 86
Link # 87
Link # 88
Link # 89
Link # 90
Link # 91
Link # 92
Link # 93
Link # 94
Link # 95
Link # 96
Link # 97
Link # 98
Link # 99
Link # 100

In [2128]:
missed_opts['Options'].value_counts()

Button    43
Toggle    20
Name: Options, dtype: int64

In [2127]:
options_df['Options'].value_counts()

Button    185
Toggle    102
Name: Options, dtype: int64

In [379]:
#options_df.to_csv('HornBach_Options_URLs.csv')

##  Toggle

In [2144]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2147]:
#df_tog = pd.DataFrame()
start = 19
end = 27
for p, idx in enumerate(missed_opts[missed_opts['Options'] == "Toggle"].index[start:end], start=start):
    
    print('Product', p)
    product_df = pd.DataFrame()
    
    url = missed_urls[idx] #all_links
    browser.get(url)
    
    # Parsing Page
    path = '//*[contains(@class, "ad-ui-Component-text_label_color_default")]'
    option_names = []

    for el in browser.find_elements_by_xpath(path):
        opt = el.text
        if 'Menge' not in opt and 'abnahme' not in opt.lower():
            option_names.append(opt)

    print(option_names)
    if any(['Durchmesser' in x for x in option_names]):
        continue

    total = 1 
    opt_lens = []
    for i in range(1, len(option_names)+1):

        path = '//*[contains(@class, "ad-ui-hide")]/div[{}]//option'.format(i)
        opts = browser.find_elements_by_xpath(path)
        if 'Menge' not in opts and 'Abnahme' not in opts:
            opt_lens.append(len(opts))

        val = len(opts)
        if val ==0:
            val=1
        total*=val
    
    product_df = pd.DataFrame()
    for i in range(total):


        # First Option
        f_idx = i//(total/opt_lens[0])+1
        path = '//*[contains(@class, "ad-ui-hide")]/div[1]//option[{}]'.format(f_idx)
        browser.find_elements_by_xpath(path)[0].click()
        opt_val_one =  browser.find_elements_by_xpath(path)[0].text
        option_name_one = option_names[0]+'-Menu'
        sleep(0.5)

        if len(opt_lens)==1:
            sleep(2)
            page_data = get_page_data()
            display(page_data)

            page_data[option_name_one] = opt_val_one
            page_data[option_name_two] = opt_val_two
            page_data = cols_dedup(page_data)
            product_df = pd.concat([product_df, page_data])

            continue

        else:
            pass

        # Second Option
        s_idx = i%opt_lens[1]+1
        path = '//*[contains(@class, "ad-ui-hide")]/div[2]//option[{}]'.format(s_idx)
        browser.find_elements_by_xpath(path)[0].click()

        opt_val_two =  browser.find_elements_by_xpath(path)[0].text
        option_name_two = option_names[1]+'-Menu'

        if len(opt_lens)==2:
            sleep(2)

            page_data = get_page_data()
            display(page_data)
            page_data[option_name_one] = opt_val_one
            page_data[option_name_two] = opt_val_two
            page_data = cols_dedup(page_data)
            product_df = pd.concat([product_df, page_data])

            continue

        else:
            pass

        # Third Option
        t_idx = i%opt_lens[2]+1
        path = '//*[contains(@class, "ad-ui-hide")]/div[3]//option[{}]'.format(t_idx)
        browser.find_elements_by_xpath(path)[0].click()

        opt_val_three =  browser.find_elements_by_xpath(path)[0].text
        option_name_three = option_names[2]+'-Menu'
        
        sleep(2)
        page_data = get_page_data()
        display(page_data)

        page_data[option_name_one] = opt_val_one
        page_data[option_name_two] = opt_val_two
        page_data[option_name_three] = opt_val_three
        page_data = cols_dedup(page_data)
        product_df = pd.concat([product_df, page_data])

        
    print('Product Table', product_df.shape)
    
    df_tog = pd.concat([df_tog, product_df])
    print('All products Table', df_tog.shape)

Product 19
['Länge', 'Durchmesser']


In [2149]:
df_tog.to_csv('HornBach_Toggle-Missed.csv')
df_tog.shape

(297, 55)

In [568]:
#df.to_csv('HornBach_Toggle.csv')

In [724]:
test = df.drop_duplicates(['SKU','URL']).reset_index(drop=True)
test.shape

(1321, 83)

## Plain Options

In [ ]:
#browser.quit()
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2166]:
df_plain_missed = pd.DataFrame()
start=0
end=71
for p, idx in enumerate(missed_opts[missed_opts['Options'].isna()].index[start:end], start=start):
    
    print('Product', p)
    product_df = pd.DataFrame()
    
    url = missed_urls[idx] #all_links
    browser.get(url)
    
    try:
        sleep(3)
        page_data = get_page_data()
    except:
        missed_products.append(p)
        print('EXCEPT EXCEPT EXCEPT EXCEPT EXCEPT', p)
        continue
    
    page_data = cols_dedup(page_data)
    display(page_data)

    df_plain_missed = pd.concat([df_plain_missed, page_data])
    df_plain_missed.to_csv('HornBach_Plain-Missed.csv')
    print('All products Table', df_plain_missed.shape)

Product 0


,Artikeltyp,Ausführung,Einsatzbereich,Material,Materialspezifizierung,Oberflächenbehandlung,Qualitätsmerkmal,Breite,Höhe,Länge,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL
0,Leiste,Sockelleiste,Innen,Holz,Fichte,Lackiert,Astfrei,18 mm,96 mm,2.400 mm,D19P,9003719072451,Sockelleiste Konsta SF380 Fichte weiß lackiert...,"13,84 € *",Art. 5491040,Materialspezifizierung: Fichte\nOberflächenbeh...,NaN,https://www.hornbach.de/shop/Sockelleiste-Kons...


All products Table (1, 18)
Product 1


,Artikeltyp,Ausführung,Einsatzbereich,Geeignet für,Stärke,Breite,Länge,Oberflächenbehandlung,Sortierung,Endlos verlegbar,Inhalt,Packinhalt,Deckfläche,Holzart,Grundfarbe,Herkunftsland,Qualität,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL
0,Profilholz,Keilspundprofil,Außen,Fassadenverkleidung,26 mm,146 mm,2.700 mm,Gehobelt,A/B-Sortierung,Nein,4 Stück,"1,58 m²","1,36 m²",Fichte,Holz,Nordeuropa,SI (sichtbarer Einbau),"25JD, 1PPD","2007005122841, 4051736187370","Keilspundprofil Fichte ""A/B"" roh 26x146x2700 mm","16,47 € *",Art. 5627616,Oberflächenbehandlung: Gehobelt\nHerkunftsland...,Beschreibung\nDie natürliche Bekleidung für Ih...,https://www.hornbach.de/shop/Keilspundprofil-F...


All products Table (2, 29)
Product 2


...

All products Table (52, 58)
Product 68


,Artikeltyp,Anwendung,Anwendungsbereich,Oberflächenbehandlung,Holzart,Dimension,Länge,Qualität,Oberfläche/Kanten,Herkunftsland,Artikelvorteil,Hinweis,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL
0,Kantholz,"Innenausbau, Neubau, Messebau","Wand, Boden, Decke, Fussboden",Gehobelt,"Fichte, Tanne",60x160 mm,4.000 mm,NSI (nicht sichtbarer Einbau),4-seitig gehobelt,Europa,"Formstabil, Verminderte Rissbildung und Verwer...","eine Keilzinkung dient dazu, Fehlstellen in ei...",FD4X,"4049868900325, 2007003147662, 2007003147655, 4...",Konstruktionsvollholz 60x160x4000 mm Fichte/Ta...,"22,85 € *",Art. 8099464,Oberfläche/Kanten: 4-seitig gehobelt,Beschreibung\nAnders als gewöhnliches Bauholz ...,https://www.hornbach.de/shop/Konstruktionsvoll...


All products Table (53, 58)
Product 69


,Artikeltyp,Dielenart/Profilart,Ausführung,Endlos verlegbar,Anwendung,Räume,Maße (LxBxS),Farbton,Grundfarbe,Material,Materialspezifizierung,Oberflächenbehandlung,Oberflächenprofil,Oberfläche Äste,Verzug,Strapazierfähigkeit/Härte,Natürliche Harzhaltigkeit,Kesseldruckimprägniert,Raumgewicht ca.,Gewicht pro Stück,Dauerhaftigkeitsklasse nach DIN EN 350-2,Verlegehinweis,Artikelvorteil,EAN,Title,Price,SKU,Minor Data,Description,URL
0,Diele,Massiv,Vollprofil,Nein,Bodenbelag,"Terrasse, Balkon",3.600 mm x 145 mm x 25 mm,Bangkirai,Braun,Holz,Bangkirai,Keine,"1-seitig genutet, 1-seitig geriffelt",Astarm,Mittel,Sehr hart,Keine,Nein,850 kg,11 kg,DKL 2,bei direkter Verschraubung ist Vorbohren empfo...,"sehr robustes, witterungsbeständiges, pflegele...","8718104304505, 2007007504157",Konsta Terrassendiele Bangkirai 25x145x3600 mm,"43,16 € *",Art. 5517585,Maße (LxBxS): 3.600 mm x 145 mm x 25 mm\nOberf...,NaN,https://www.hornbach.de/shop/Konsta-Terrassend...


All products Table (54, 70)


In [2092]:
url = 'https://www.hornbach.de/shop/Konstruktionsvollholz-60x80x4000-mm-Fichte-Tanne-NSI-nicht-sichtbarer-Einbau/8099458/artikel.html'
url in all_links#[0]

False

In [773]:
df_plain = df_plain.reset_index(drop=True)

In [2089]:
df_plain[df_plain['SKU'].str.contains('8099458')].shape

(0, 154)

In [776]:
print(df_plain['URL'].loc[897])

https://www.hornbach.de/shop/Konsta-Terrassendiele-Fichte-KDI-32x132x2400-mm/7833741/artikel.html


## Buttons

In [2161]:
#browser.quit()
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2150]:
missed_opts[missed_opts['Options'] == "Button"].shape

(43, 2)

In [2163]:
%%time
start = 28
end = 44
#df_but_missed = pd.DataFrame()
for p, idx in enumerate(missed_opts[missed_opts['Options'] == "Button"].index[start:end], start=start):
    
    print('Product', p)
    product_df = pd.DataFrame()
    
    url = missed_urls[idx] #all_links
    browser.get(url)
    
    # Parsing Page
    path = '//*[contains(@class, "ad-ui-Component-text_label_color_default")]'
    option_names = []

    for el in browser.find_elements_by_xpath(path):
        opt = el.text
        if 'Menge' not in opt and 'abnahme' not in opt.lower():
            option_names.append(opt)
    if any(['Durchmesser' in x for x in option_names]):
        continue
    sleep(2)
    
    # Choosing Options
    path = '//*[contains(@data-selenium-test, "selectionButton")]'
    opts = browser.find_elements_by_xpath(path)
    for n in range(len(opts)):
        sleep(2)
        path = '//*[contains(@data-selenium-test, "selectionButton")]'
        browser.find_elements_by_xpath(path)[n].click()
        sleep(1)
        
        opt_val =  browser.find_elements_by_xpath(path)[n].text
        option_name = option_names[0]+'-Menu'
        
        # Getting Data
        sleep(2)
        try:
            page_data = get_page_data()
        except:
            missed_products.append(p)
            print('EXCEPT EXCEPT EXCEPT EXCEPT EXCEPT', p)
            continue
        
        
        
        page_data[option_name] = opt_val
        
        page_data = cols_dedup(page_data)
        display(page_data)
    
        df_but_missed = pd.concat([df_but_missed, page_data])
        df_but_missed.to_csv('HornBach_Button-Missed.csv')
        print('All products Table', df_but_missed.shape)

Product 28


,Artikeltyp,Anwendung,Anwendungsbereich,Oberflächenbehandlung,Holzart,Stärke,Breite,Länge,Oberfläche/Kanten,Herkunftsland,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL,Länge-Menu
0,Latte,"Innenausbau, Messebau, Neubau","Wand, Boden, Decke, Fussboden",Keine,Fichte,24 mm,48 mm,2.000 mm,Sägerau,Europa,DDWX,"4260031140106, 2007003126070, 2001000664009, 4...",Latte 24x48x2000 mm Fichte/Tanne,"0,76 € *",Art. 1000664,Oberfläche/Kanten: Sägerau,NaN,https://www.hornbach.de/shop/Latte-24x48x2000-...,2.000 mm


All products Table (97, 61)


,Artikeltyp,Anwendung,Anwendungsbereich,Oberflächenbehandlung,Holzart,Stärke,Breite,Länge,Oberfläche/Kanten,Herkunftsland,Hinweis,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL,Länge-Menu
0,Latte,"Innenausbau, Messebau, Neubau","Wand, Boden, Decke, Fussboden",Keine,Fichte,24 mm,48 mm,2.500 mm,Sägerau,Europa,Ist dieser Artikel in Ihrem Markt nicht am Lag...,73GC,"4260031140199, 2001000675005, 2007003126087, 4...",Latte 24x48x2500 mm Fichte/Tanne,"0,96 € *",Art. 1000675,Oberfläche/Kanten: Sägerau,NaN,https://www.hornbach.de/shop/Latte-24x48x2500-...,2.500 mm


All products Table (98, 61)


...

All products Table (137, 66)


,Bestellhinweis,Artikeltyp,Ausführung,Variante,Einsatzbereich,Anwendung,Anwendungsbereich,Räume,Eigenschaft,Material,Oberflächenbehandlung,Länge,Breite,Stärke,Norm,Emissionsklasse,Kantenausprägung,Grundfarbe,Gewicht pro Stück,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL,Stärke-Menu
0,Ist dieser Artikel in Ihrem Markt nicht am Lag...,Ausbauplatte,Verlegeplatte,OSB-Verlegeplatte,"Innen, Feuchtraum","Bodenbelag, Innenausbau, Ladenbau, Messebau, S...","Wand, Boden, Decke","Dachboden, Gartenhaus, Werkstatt",Formstabil,Holzwerkstoff,Keine,2.050 mm,625 mm,22 mm,"EN 13986, EN 300",E 1,4-seitig mit Nut und Feder,Holz,17 kg,EPP6,"5450454466337, 2007003710927, 2007005942586, 8...",OSB3 Verlegeplatte 22x625x2050 mm E1 ungeschli...,"7,27 € *",Art. 8367971,Stärke: 22 mm\nBreite: 625 mm\nEmissionsklasse...,Beschreibung\nSie brauchen für den Rohbau oder...,https://www.hornbach.de/shop/OSB3-Verlegeplatt...,22 mm


All products Table (138, 66)


,Bestellhinweis,Artikeltyp,Ausführung,Variante,Einsatzbereich,Anwendung,Anwendungsbereich,Räume,Eigenschaft,Material,Oberflächenbehandlung,Länge,Breite,Stärke,Norm,Emissionsklasse,Kantenausprägung,Grundfarbe,Gewicht pro Stück,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL,Stärke-Menu
0,Ist dieser Artikel in Ihrem Markt nicht am Lag...,Ausbauplatte,Verlegeplatte,OSB-Verlegeplatte,"Innen, Feuchtraum","Bodenbelag, Innenausbau, Ladenbau, Messebau, S...","Wand, Boden, Decke","Dachboden, Gartenhaus, Werkstatt",Formstabil,Holzwerkstoff,Keine,2.050 mm,625 mm,25 mm,"EN 13986, EN 300",E 1,4-seitig mit Nut und Feder,Holz,19 kg,UEDC,"5450454466320, 2007004583353, 8591000043194",OSB3 Verlegeplatte 25x625x2050 mm E1 ungeschli...,"9,21 € *",Art. 8762391,Stärke: 25 mm\nBreite: 625 mm\nEmissionsklasse...,Beschreibung\nSie brauchen für den Rohbau oder...,https://www.hornbach.de/shop/OSB3-Verlegeplatt...,25 mm


All products Table (139, 66)
CPU times: user 5.99 s, sys: 615 ms, total: 6.61 s
Wall time: 15min 48s


### Getting Data Functions

In [2165]:
def get_page_data():
    # Title
    path = '//*[@id="product-information"]/h1'
    title = browser.find_elements_by_xpath(path)[0].text
    
    # Price
    path = '//*[contains(@class, "price_main")]'
    price = browser.find_elements_by_xpath(path)[0].text
    
    # SKU
    path = '//*[contains(@class, "articleNumber")]'
    sku = browser.find_elements_by_xpath(path)[0].text
    
    # Minor Data
    path = '//*[contains(@class, "Component-unordered_list")]'
    minor_data = browser.find_elements_by_xpath(path)[0].text
    
    # Show More Data
    actions = ActionChains(browser)
    for _ in range(6):
        actions.send_keys(Keys.PAGE_DOWN)
        actions.perform()
        sleep(0.1)
    sleep(1)
    for _ in range(4):
        actions.send_keys(Keys.PAGE_UP)
        actions.perform()
        sleep(0.1)

    sleep(2)
    try:
        path = '//*[@data-selenium-test="attributes-show-more-button"]'
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, path)))
        sleep(1)
        element.click()
    except:
        pass
    
    #sleep(1)
    # Description
    try:
        path = '//*[@id="details"]'
        descr = browser.find_elements_by_xpath(path)[0].text
    except:
        descr=np.nan
        pass
    
    sleep(2)
    # Main Data
    cols = []
    values = []
    path = '//*[@id="attribute"]//li' #-key
    for el in browser.find_elements_by_xpath(path):
        col = el.text.split('\n')[0]
        value = el.text.split('\n')[-1]
        cols.append(col)
        values.append(value)
    page_data = pd.DataFrame([values], columns=cols)
    
    page_data['Title'] = title
    page_data['Price'] = price
    page_data['SKU'] = sku
    page_data['Minor Data'] = minor_data
    page_data['Description'] = descr
    page_data['URL'] = browser.current_url
    '''
    sleep(2)
    
    actions = ActionChains(browser)
    for _ in range(3):
        actions.send_keys(Keys.PAGE_DOWN)
        actions.perform()
        sleep(0.1)
    
    sleep(1)
    for _ in range(9):
        actions.send_keys(Keys.PAGE_UP)
        actions.perform()
        sleep(0.1)
    sleep(2)
    '''
    return page_data

In [503]:
#Check Dupl Columns
def cols_dedup(data):
    cols = []
    dupl_cols = []
    dupl_idx = []

    for i, col in enumerate(data.columns):
        if col not in cols:
            cols.append(col)
        else:
            dupl_cols.append(col)
            dupl_idx.append(i)


    if dupl_cols:
        new_cols = data.columns.to_list()
        count=0
        for idx in dupl_idx:
            count+=1
            new_cols[idx] = new_cols[idx]+'-'+str(count)
        data.columns = new_cols
    return data

# Data Processing

In [2522]:
df_proc = pd.concat([df_plain, df, df_but,
                     df_plain_missed, df_but_missed, df_tog]).reset_index(drop=True)
df_proc.shape

(4778, 201)

### Combine all columns to mine text form

In [2523]:
df_proc['Combined'] = ''
for col in df_proc.columns:
    df_proc['Combined'] = df_proc['Combined'] + '; ' + df_proc[col].astype(str)

In [2524]:
df_proc['Combined'] = df_proc['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

###  Quality

In [2545]:
df_proc['Quality'] = ''
for i in df_proc.index:
    df_proc['Quality'].loc[i] = []
    
    for qual in keywords['Quality'].unique():
        qual = str(qual) 
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        
        if qual!= 'nan' and len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(df_proc['Quality'].loc[i]):
            df_proc['Quality'].loc[i].append(qual)
            
    try:
        df_proc['Quality'].loc[i] = ', '.join(df_proc['Quality'].loc[i])
    except:
        pass
df_proc['Quality'] =  df_proc['Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
df_proc['Quality'] = df_proc['Sortierung'].fillna(df_proc['Quality'])

df_proc['Quality'].value_counts()

B/C                     68
A/B                     59
A-Sortierung            34
U/S hobelfallend        10
A/B-Sortierung           9
GK III, B-Sortierung     9
B-Sortierung             7
B/BB                     7
BB/WG                    7
Unsortiert               7
1. Wahl                  4
C-Sortierung             4
A/B, B/C                 1
Name: Quality, dtype: int64

### Sorting Class

In [2546]:
sort_classes = ['S7', 'S10', 'S13', 'C16M', 'C24M', 'C30M', 'D30', 'D35', 'D40',
               'D60', 'D70', 'geringe tragfähigkeit', 'mittlere tragfähigkeit',
               'hohe tragfähigkeit']

In [2547]:
df_proc['Sorting Class'] = ''
for i in df_proc.index:
    df_proc['Sorting Class'].loc[i] = []
    for sort_class in sort_classes:
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(df_proc['Sorting Class'].loc[i]):
            df_proc['Sorting Class'].loc[i].append(sort_class)
       
    try:
        df_proc['Sorting Class'].loc[i] = ', '.join(df_proc['Sorting Class'].loc[i])
    except:
        pass
df_proc['Sorting Class'] =  df_proc['Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
#df_proc['Sorting Class'] =  df_proc['Sorting Class'].fillna(df_proc['Sortierklasse nach DIN 4074']).fillna(df_proc['Festigkeitsklasse nach EN 338'])
df_proc['Sorting Class'].value_counts()

S10    4
Name: Sorting Class, dtype: int64

## Main parameters

### Wood Types

In [2548]:
woods = ['Ahorn', 'Tanne', 'Sperrholz', 'Nussbaum', 'Erle',
        'Bongossi (Azobé)', 'Birke', 'Buche', 'Douglasie',
        'Edelkastanie', 'Eiche', 'Elsbeere', 'Erle', 'Esche', 'Fichte',
        'Kiefer', 'Kirschbaum', 'Lärche', 'Linde', 'Mahagoni', 'Teak',
        'Pappel', 'Robinie', 'Tanne', 'Ulme', 'Walnuss', 'Weide', 'Birne',
        'Fichte/Tanne', 'Fichte/Kiefer']

In [2549]:
df_proc['Meta Wood'] = ''
for i in df_proc.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0:
            df_proc['Meta Wood'].loc[i] = wood

In [2550]:
df_proc['Meta Wood'] = df_proc['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [2551]:
for col in df_proc.columns:
    if 'mater' in col.lower() or 'holz' in col.lower():
        print(col)

Material
Materialspezifizierung
Holzart
Materialstärke
Materialspezifikation
Bauplatte, Bauholz, Carport, Dachüberstand, Schalung, Sperrholz
Holzherkunft
Material Gehäuse
Material Rad
Austretende Holzinhaltsstoffe
Material, Behandlung
Holz


In [2552]:
df_proc['Wood Type'] = df_proc['Meta Wood'].fillna(df_proc['Material'])
df_proc['Wood Type'] = df_proc['Wood Type'].fillna(df_proc['Materialspezifizierung'])
df_proc['Wood Type'] = df_proc['Wood Type'].fillna(df_proc['Holzart'])
df_proc['Wood Type'] = df_proc['Wood Type'].fillna(df_proc['Materialspezifikation'])

In [2553]:
df_proc['Wood Type'] = df_proc['Wood Type'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [2554]:
df_proc['Wood Type'].isna().sum()

40

In [2555]:
df_proc['Wood Type'].value_counts()

Fichte                  917
Kunststoff              862
Kiefer                  702
Fichte/Kiefer           390
Fichte/Tanne            364
Holzwerkstoff           284
Holz                    243
Buche                   197
Metall                  146
Birke                   100
Nussbaum                 96
Linde                    83
Esche                    56
Eiche                    53
Pappel                   34
Stahl                    28
Douglasie                25
Ahorn                    23
Lärche                   21
Metall, Verzinkt         17
Eisen                    16
Tanne                    13
Aluminium                13
Messing                  10
PVC                       9
Verbundwerkstoff          7
Walnuss                   6
Beton                     5
Holzfaser                 4
Kunststoff, Metall        4
Gummi                     2
Edelstahl                 1
Stahl verzinkt            1
Holz, Stahl               1
Teak                      1
Normalbeton         

#### Cheking wrong typs and woods

In [2556]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
              'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl','Metall, Verzinkt','Kunststoff, Metall',
             'Verbundwerkstoff','Beton','Edelstahl','Normalbeton','Chrom-Vanadium-Stahl','Holz, Stahl',
             'Stahl verzinkt']

In [2557]:
# Percentage of wrong wood types
df_proc[(df_proc['Wood Type'].isin(wrong_wood))].shape[0]/df_proc.shape[0]*100

22.959397237337797

In [2558]:
df_proc[(df_proc['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

Kunststoff              862
Metall                  146
Stahl                    28
Metall, Verzinkt         17
Aluminium                13
PVC                       9
Verbundwerkstoff          7
Beton                     5
Kunststoff, Metall        4
Metall, Kunststoff        1
Holz, Stahl               1
Normalbeton               1
Stahl verzinkt            1
Chrom-Vanadium-Stahl      1
Edelstahl                 1
Name: Wood Type, dtype: int64

In [2559]:
df_proc.shape

(4778, 206)

In [2560]:
df_proc = df_proc[(~df_proc['Wood Type'].isin(wrong_wood))]
df_proc.shape

(3681, 206)

In [2561]:
df_proc['Wood Type'].value_counts()

Fichte           917
Kiefer           702
Fichte/Kiefer    390
Fichte/Tanne     364
Holzwerkstoff    284
Holz             243
Buche            197
Birke            100
Nussbaum          96
Linde             83
Esche             56
Eiche             53
Pappel            34
Douglasie         25
Ahorn             23
Lärche            21
Eisen             16
Tanne             13
Messing           10
Walnuss            6
Holzfaser          4
Gummi              2
Mahagoni           1
Teak               1
Name: Wood Type, dtype: int64

In [2562]:
#df_proc['Wood Type'] = df_proc['Wood Type'].str.lower()

### Surface

In [2563]:
strong_surfaces = ['geschliffen', 'gehobelt', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 
                   'deckend', 'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig']

weak_surfaces = ['roh', 'keine']

In [2564]:
df_proc['Combined Surface Treat'] = ''
for i in df_proc.index:
    df_proc['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(df_proc['Combined Surface Treat'].loc[i]):
            df_proc['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in df_proc['Combined'].loc[i].lower() and surface.lower() not in ', '.join(df_proc['Combined Surface Treat'].loc[i]):
            df_proc['Combined Surface Treat'].loc[i].append(surface)
    try:
        df_proc['Combined Surface Treat'].loc[i] = ', '.join(df_proc['Combined Surface Treat'].loc[i])
    except:
        pass

In [2565]:
df_proc['Combined Surface Treat'] = df_proc['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )
df_proc['Surface Treatment'] = df_proc['Oberflächenbehandlung']

In [2566]:
cols = ['Oberfläche','Oberfläche/Kanten','Oberflächenprofil', 'Combined Surface Treat']
for col in cols:
    df_proc['Surface Treatment'] = df_proc['Surface Treatment'].fillna(df_proc[col])

    df_proc['Surface Treatment'] = df_proc['Surface Treatment'].str.lower()
df_proc['Surface Treatment'].value_counts()

roh                                                     1497
gehobelt                                                1004
foliert                                                  344
unbehandelt                                              300
keine                                                    125
lackiert                                                  68
gehobelt, unbehandelt                                     52
melaminharzbeschichtet                                    32
keine, gehobelt                                           25
grundiert                                                 24
phenolharzbeschichtet                                     18
geölt                                                     16
glatt                                                     16
gehobelt, 4-seitig gehobelt                               12
bedruckt                                                  10
imprägniert                                                8
geschliffen             

### Surface Quality

In [2567]:
surf_quality = ['NSI', 'SI']

In [2568]:
df_proc['Surface Quality'] = ''
for i in df_proc.index:
    df_proc['Surface Quality'].loc[i] = []
    for surf_qual in surf_quality:
        pat = r' {} |^{} | {}$|[a-z] {}\;'.format(surf_qual.lower(), surf_qual.lower(), surf_qual.lower(), surf_qual.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and surf_qual.lower() not in ', '.join(df_proc['Surface Quality'].loc[i]):
            df_proc['Surface Quality'].loc[i].append(surf_qual)
            
    try:
        df_proc['Surface Quality'].loc[i] = ', '.join(df_proc['Surface Quality'].loc[i])
    except:
        pass
df_proc['Surface Quality'] =  df_proc['Surface Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
#df_proc['Surface Quality'] =  df_proc['Surface Quality'].fillna(df_proc['Oberflächenqualität'])

In [2569]:
df_proc['Surface Quality'].value_counts()

SI     704
NSI    256
Name: Surface Quality, dtype: int64

### Drying method

In [2664]:
df_proc['Combined Drying Method'] = ''
for i in df_proc.index:
    df_proc['Combined Drying Method'].loc[i] = []
    for method in keywords['Drying Method'].unique():
        method = str(method) 
        
        pat = r' {} |^{} | {}$| {}\;'.format(method.lower(), method.lower(), method.lower(), method.lower())
        if method!='nan' and len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and method.lower() not in ', '.join(df_proc['Combined Drying Method'].loc[i]):
            df_proc['Combined Drying Method'].loc[i].append(method)

    try:
        df_proc['Combined Drying Method'].loc[i] = ', '.join(df_proc['Combined Drying Method'].loc[i])
    except:
        pass

In [2665]:
df_proc['Combined Drying Method'] =  df_proc['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [2666]:
df_proc['Drying Method'] = df_proc['Combined Drying Method']

In [2667]:
df_proc['Drying Method'].value_counts()

frisch    2
Name: Drying Method, dtype: int64

## Dimentions

In [2576]:
df_proc[['Länge','Stärke','Breite']].isna().sum()/df_proc.shape[0]*100

Länge      4.618310
Stärke    69.084488
Breite    31.105678
dtype: float64

### Length

In [2577]:
df_proc['Länge'] = df_proc['Länge'].apply(lambda x: np.nan if x=='' else x)

In [2578]:
df_proc['Länge in mm'].isna().sum()/df_proc.shape[0]*100

99.1306710133116

In [2579]:
df_proc['Länge'] = df_proc['Länge'].fillna(df_proc['Länge in mm'])
df_proc['Länge'].isna().sum()

138

In [2580]:
df_proc[df_proc['Länge'].isna()]['Material'].value_counts()

Holz    78
Name: Material, dtype: int64

In [2581]:
# Getting non-null columns
import math
def get_nonnan_cols(main_idx):
    non_nan_cols = []
    for idx, i in enumerate(df_proc.loc[main_idx]):
        try:
            if math.isnan(i)==False:
                col = df_proc.loc[main_idx].index[idx]
                non_nan_cols.append(col)
        except:
            col = df_proc.loc[main_idx].index[idx]
            non_nan_cols.append(col)
            pass
    return non_nan_cols

In [2582]:
df_proc[(df_proc['Länge'].isna()) &
        (~df_proc['Title'].str.contains('mm'))].shape

(13, 191)

In [2583]:
df_proc[(df_proc['Länge'].isna()) &
        (~df_proc['Maße (LxBxS)'].isna())].shape

(54, 191)

#### Title Dims

In [2584]:
title_dims = df_proc['Title'].str.lower().str.extract(r'(\d.+(?= mm))|(\d.+(?=mm))', expand=True)
title_dims.columns = ['First Ext', 'Second Ext']
title_dims['First Ext'] = title_dims['First Ext'].fillna(title_dims['Second Ext'])

In [2585]:
title_dims['Final Ext'] = title_dims['First Ext'].str.extract(r'(\d+x\d+x\d+)')
title_dims['Final Ext'].isna().sum()

273

In [2586]:
title_dims['Title Lange'] = np.nan
title_dims['Title Breite'] = np.nan 
title_dims['Title Stärke'] = np.nan 

In [2587]:
for idx in title_dims.index:
    try:
        dims = [int(x) for x in title_dims['Final Ext'].loc[idx].split("x")]
    except:
        continue
        
    # L
    try:
        title_dims['Title Lange'].loc[idx] = np.max(dims)
    except:
        pass
    
    # S
    try:
        title_dims['Title Stärke'].loc[idx] = np.min(dims)
    except:
        pass
    
    # B
    try:
        title_dims['Title Breite'].loc[idx] = np.median(dims)
    except:
        pass

In [2588]:
df_proc['Länge'] = df_proc['Länge'].fillna(title_dims['Title Lange'])
df_proc['Länge'].isna().sum()/df_proc.shape[0]*100, df_proc['Länge'].isna().sum()

(0.9779951100244498, 36)

### Breite

In [2589]:
#df_proc[df_proc['Title'].str.contains('Rechteckleiste')].sample()

In [2590]:
df_proc['Breite'] = df_proc['Breite'].apply(lambda x: np.nan if x=='' else x)

In [2591]:
def get_isna(data, col):
    print(data[col].isna().sum()/data.shape[0]*100)

In [2592]:
get_isna(df_proc, 'Breite')

31.10567780494431


In [2593]:
df_proc['Breite'] = df_proc['Breite'].fillna(df_proc['Breite in mm'])
df_proc['Breite'] = df_proc['Breite'].fillna(title_dims['Title Breite'])
get_isna(df_proc, 'Breite')

3.124151045911437


### Starke

In [2594]:
df_proc['Stärke'] = df_proc['Stärke'].fillna(df_proc['Höhe in mm'])
df_proc['Stärke'] = df_proc['Stärke'].fillna(df_proc['Höhe'])
df_proc['Stärke'] = df_proc['Stärke'].fillna(df_proc['Tiefe'])
df_proc['Stärke'] = df_proc['Stärke'].fillna(df_proc['Tiefe in mm'])

In [2595]:
df_proc[~df_proc['Tiefe'].isna()].shape

(47, 191)

In [2596]:
df_proc['Stärke'] = df_proc['Stärke'].apply(lambda x: np.nan if x=='' else x)

In [2597]:
get_isna(df_proc, 'Stärke')

29.25835370823146


In [2598]:
df_proc['Stärke'] = df_proc['Stärke'].fillna(title_dims['Title Stärke'])
get_isna(df_proc, 'Stärke')

3.4773159467535995


In [2599]:
df_proc['Stärke'].isna().sum()

128

In [2600]:
main_idx = df_proc[(df_proc['Stärke'].isna()) 
                  # & (df_proc['Title'].str.contains('mm'))
                  ].sample().index[0]

df_proc.loc[main_idx:main_idx][get_nonnan_cols(main_idx)]

,Artikeltyp,Länge,Title,Price,SKU,Minor Data,Description,URL,,Benötigtes Zubehör,Aufstellbreite,Aufstelltiefe,Außenbreite inkl. Dachüberstand,Außentiefe inkl. Dachüberstand,Außenbreite,Gefälle,Schneelast,Im Lieferumfang enthalten,Combined,Meta Wood,Wood Type,Combined Surface Treat,Surface Treatment,Combined Drying Method,Drying Method
863,Bausatz,450 cm,Holzkonstruktion Satteldach 350 x 450 cm,"3285,04 € *",Art. 4690660,"Pfostenstärke: 13 x 13 cm\nSchneelast: 3,923 k...",Beschreibung\nRobuste Holzkonstruktion mit Sat...,https://www.hornbach.de/shop/Holzkonstruktion-...,,"Werkzeug, Leiter",410 cm,540 cm,410 cm,540 cm,350 cm,Seitlich,"3,923 kN/m²","Balken aus Konstruktionsvollholz, Aufbauplan, ...",Bausatz; 450 cm; Holzkonstruktion Satteldach 3...,Fichte,Fichte,"gehobelt, unbehandelt","gehobelt, unbehandelt",technisch getrocknet,technisch getrocknet


In [2602]:
starke_cols = ['Stärke', 'Höhe', 'Höhe in mm','Tiefe', 'Tiefe in mm','Abmessungen (LxTxS)']
df_proc[~df_proc['Tiefe'].isna()][starke_cols].sample(2)

,Stärke,Höhe,Höhe in mm,Tiefe,Tiefe in mm,Abmessungen (LxTxS)
144,58 mm,NaN,NaN,58 mm,NaN,NaN
215,28 mm,NaN,NaN,28 mm,NaN,NaN


In [2603]:
dim_cols = ['Stärke','Breite','Länge', 'Höhe', 'Höhe in mm','Maße (BxHxT)', 'Tiefe', 'Tiefe in mm','Maße',
             'Länge in cm','Breite in mm', 'Maße (LxBxS)','Maß (B x H)','Abmessungen (LxTxS)']

#### Cleaning NaN Dims

In [2604]:
df_proc.shape

(3681, 191)

In [2605]:
df_proc = df_proc[(~df_proc['Breite'].isna()) & 
                  (~df_proc['Stärke'].isna()) & 
                  (~df_proc['Länge'].isna())].reset_index(drop=True)
df_proc.shape

(3527, 191)

### To Numeric

#### B

In [2606]:
print(df_proc['URL'].loc[682])

https://www.hornbach.de/shop/Endleiste-Balsa-beidseitig-konisch-10x50x1000-mm/6178702/artikel.html


In [2607]:
df_proc[df_proc['Breite'].astype(str).str.contains(r'^\d\.')]['Breite'].sample(3)

413     1.106 mm
2686    1.550 mm
2828    2.500 mm
Name: Breite, dtype: object

In [2608]:
df_proc['Breite'] = df_proc['Breite'].astype(str).str.replace(r'mm| ','')
df_proc['Breite'] = df_proc['Breite'].astype(str).str.replace(r'\,','.')
df_proc['Breite'] = df_proc['Breite'].astype(str).str.replace(r'(?<=^\d)\.','')

In [2609]:
df_proc[df_proc['Breite'].astype(str).str.contains(r'cm')].shape

(69, 191)

In [2610]:
df_proc['Breite'] = df_proc['Breite'].apply(lambda x: np.nan if type(x)==str and 'cm' in x else x)

In [2611]:
df_proc['Breite'] = df_proc['Breite'].astype(float)

#### L

In [2612]:
df_proc[df_proc['Länge'].astype(str).str.contains(r'\D')].shape

(3527, 191)

In [2613]:
df_proc[df_proc['Länge'].astype(str).str.contains(r'cm')].shape

(69, 191)

In [2614]:
df_proc['Länge'] = df_proc['Länge'].apply(lambda x: np.nan if type(x)==str and 'cm' in x else x)

In [2615]:
df_proc[df_proc['Breite'].astype(str).str.contains(r'\D')].sample()

,Artikeltyp,Ausführung,Einsatzbereich,Anwendung,Anwendungsbereich,Räume,Geeignet für,Stärke,Breite,Länge,Material,Materialspezifizierung,Oberflächenbehandlung,Endlos verlegbar,Inhalt,Packinhalt,Deckfläche,Grundfarbe,Dekoroptik,Hinweis,Artikelkurznummer (AKN),EAN,Title,Price,SKU,Minor Data,Description,URL,Sortierung,Holzart,Herkunftsland,Qualität,Nutwangenstärke,,Tiefe,Qualitätsmerkmal,Höhe,Kompatibel mit,Farbe,Geeignet für Untergrund,Eigenschaften,Lieferanten-Artikelnummer,Farbton,Montageart,Kabelführung,Breite in mm,Länge in mm,Eigenschaft,Brandschutzklasse,"Starke Anfangshaftung, Hohe Endfestigkeit, Lösemittelfrei",Hinweise Allg. oder zur Reichweite,Variante,Maß a,Maß b,Struktur,Materialstärke,Höhe in mm,Benötigtes Zubehör,Form,4012230072395,Materialspezifikation,Gewicht,Art der Verleimung,Keilgezinkte Lamellen,"Bauplatte, Bauholz, Carport, Dachüberstand, Schalung, Sperrholz",Oberfläche,Kantenausprägung,Gewicht pro Stück,Oberfläche/Kanten,Befestigungssystem,600 mm,Maße (BxHxT),Holzherkunft,Durchmesser 1,Norm,Design,Tiefe in mm,Artikelvorteil,Maße,Material Gehäuse,Material Rad,Rückenloch,Raddurchmesser,Bauhöhe,Ausladung,Radbreite,Stift Durchmesser,Max. Tragkraft,Ausführung Lager,Gewicht in kg,Nutzung,Einband,Erscheinungsdatum,Auflage,Seitenanzahl,Hinweis Zuschnitt,Länge in cm,Dimension,Aufstellbreite,Aufstelltiefe,Außenbreite inkl. Dachüberstand,Außentiefe inkl. Dachüberstand,Außenbreite,Innenbreite,Durchfahrtshöhe vorn,Durchfahrtshöhe hinten,Kesseldruckimprägniert,Pfostenstärke,Dachform,Dachfläche,Dachstärke,Gefälle,Schneelast,Im Lieferumfang enthalten,Serie,Art der Imprägnierung,"Wand, Boden, Decke",Emissionsklasse,Bestellhinweis,Dielenart/Profilart,Verzug,Strapazierfähigkeit/Härte,Natürliche Harzhaltigkeit,Oberflächenprofil,Oberfläche Äste,Raumgewicht ca.,Maße (LxBxS),Verwendungszweck,Austretende Holzinhaltsstoffe,Durchmesser,Bohrdurchmesser,Inhalt-1,Antrieb,Behandlung,Deckbreite,Flächengewicht,Hinweis zur Farbe,Oberflächenstruktur,Hinweis zur Verarbeitung,Länge-Menu,Breite-Menu,Höhe-Menu,Durchmesser-Menu,Durchmesser in mm,Gewicht in g,Breite in mm-Menu,Durchmesser in mm-Menu,DIN / EN / ISO,Oberfläche-/Farbe,Kopfform,Gewindeart,Gewinde-Typ,d,l,Anzahl Lagen,Stärke-Menu,Passend für Dekor,Farbe-Menu,"Material, Behandlung",Maß (B x H),Dimension-Menu,Inhalt-Menu,Artikelvorteil-Menu,Thema,Unsortiert,384 mm,Tiefe in mm-Menu,Farbbereich,Abmessungen (LxTxS),Kante vorne,Tiefe-Menu,Dauerhaftigkeitsklasse nach DIN EN 350-2,Verlegehinweis,Länge in mm-Menu,Modellbau,DIN 4102 B2,800 mm,1.200 mm,2.000 mm,Holz,Polystyrolglas,Combined,Quality,Sorting Class,Meta Wood,Wood Type,Combined Surface Treat,Surface Treatment,Surface Quality,Combined Drying Method,Drying Method
2600,Kantholz,NaN,NaN,"Innenausbau, Messebau, Neubau","Wand, Boden, Decke, Fussboden, Terrasse",NaN,NaN,80,240.0,5.000 mm,NaN,NaN,Gehobelt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P9ZX,"4306517249335, 2007005554307",Brettschichtholz 80x240x5000 mm Fichte SI (sic...,"94,55 € *",Art. 5955523,"Oberfläche/Kanten: Gefast, 4-seitig gehobelt","Beschreibung\nBrettschichtholz, auch Leimbinde...",https://www.hornbach.de/shop/Brettschichtholz-...,NaN,Fichte,Europa,SI (sichtbarer Einbau),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Gefast, 4-seitig gehobelt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"keilgezinkt, formstabil, verminderte Rissbildu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80x240 mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000 mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80x240 mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kantholz; Innenausbau, Messebau, Neubau; Wand,...",NaN,NaN,Fichte,Fichte,"gehobelt, gefast, 4-Seitig gehobelt",gehobelt,SI,NaN,NaN


In [2616]:
df_proc['Länge'] = df_proc['Länge'].astype(str).str.replace(r'mm| ','')
df_proc['Länge'] = df_proc['Länge'].astype(str).str.replace(r'\,','.')
df_proc['Länge'] = df_proc['Länge'].astype(str).str.replace(r'(?<=^\d)\.','')

In [2617]:
df_proc[df_proc['Länge'].astype(str).str.contains(r'm')].shape

(12, 191)

In [2618]:
df_proc['Länge'] = df_proc['Länge'].apply(lambda x: np.nan if type(x)==str and 'm' in x else x)

In [2619]:
df_proc['Länge'] = df_proc['Länge'].astype(float)

#### S

In [2620]:
df_proc[df_proc['Stärke'].astype(str).str.contains(r'^\d\.')]['Stärke'].sample(3)

3094    5
321     3
3096    5
Name: Stärke, dtype: object

In [2621]:
df_proc['Stärke'] = df_proc['Stärke'].astype(str).str.replace(r'mm| ','')
df_proc['Stärke'] = df_proc['Stärke'].astype(str).str.replace(r'\,','.')

In [2622]:
df_proc[df_proc['Stärke'].astype(str).str.contains(r'cm')].shape

(69, 191)

In [2623]:
df_proc['Stärke'] = df_proc['Stärke'].apply(lambda x: np.nan if type(x)==str and 'cm' in x else x)
df_proc['Stärke'] = df_proc['Stärke'].astype(float)

In [2624]:
df_proc = df_proc[(~df_proc['Breite'].isna()) & 
                  (~df_proc['Stärke'].isna()) & 
                  (~df_proc['Länge'].isna())].reset_index(drop=True)
df_proc.shape

(3446, 191)

### Price

In [2625]:
df_proc['Price'] = df_proc['Price'].str.replace('\,','.')
df_proc['Price'] = df_proc['Price'].str.replace(r'[^\.\d]','')
df_proc['Price'] = df_proc['Price'].astype(float)

### SKU

In [2626]:
df_proc['SKU'] = df_proc['SKU'].str.replace(r'[^\d]','')

In [2627]:
df_proc[df_proc['SKU'].str.contains('8099459')].shape

(1, 191)

### Images

In [ ]:
#browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2031]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':np.nan}, index=[df_proc.index])
images_df.shape

(3034, 2)

In [2496]:
img_urls_missed = sheets_clean[sheets_clean['Images'].isna()]['URL'].unique()

In [2497]:
images_df_missed = pd.DataFrame({'Images':np.nan, 'URL':np.nan}, index=range(len(img_urls_missed)))
images_df_missed.head(1)

,Images,URL
0,NaN,NaN


In [2484]:
len(missed_urls)

133

In [2498]:
#browser.quit()
start=0
for p, idx in enumerate(range(len(img_urls_missed)), start=start):
    
    print('Product', p)
    
    
    #url = df_proc['URL'].loc[idx]
    url = img_urls_missed[idx]
    browser.get(url)
    sleep(1)
    
    img_urls = []
    path = '//*[contains(@class, "ad-ui-innerBar")]//div'
    img_num = len(browser.find_elements_by_xpath(path))

    for i in range(1,img_num+1):

        path = '//*[contains(@class, "ad-ui-innerBar")]/div[{}]'.format(i)
        try:
            browser.find_elements_by_xpath(path)[0].click()
            sleep(1)
            path = '//*[contains(@class, "awssld__content")]/img'
            img_url = browser.find_elements_by_xpath(path)[0].get_attribute('src')
            print(img_url)
            img_urls.append(img_url)
        except:
            pass
    
    images_df_missed['URL'].loc[idx]=url
    images_df_missed['Images'].loc[idx]= '; '.join(img_urls)

Product 0
https://cdn.hornbach.de/data/shop/D04/001/780/491/455/369/DV_8_6851427_01_4c_CH_20181019151656.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/455/372/DV_8_6851427_02_4c_CH_20181019151656.jpg
Product 1
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/678/DV_8_5640459_03_4c_DE_20190128134758.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/677/DV_8_5640459_02_4c_DE_20190128134758.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/679/DV_8_5640459_04_4c_DE_20190128134758.jpg
Product 2
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/676/DV_8_3267547_03_4c_DE_20190128141651.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/675/DV_8_3267547_02_4c_DE_20190128141651.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/674/DV_8_3267547_01_4c_DE_20190128141651.jpg
Product 3
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/682/DV_8_3476355_03_4c_DE_20190128134758.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/378/681/DV_8_347635

https://cdn.hornbach.de/data/shop/D04/001/700/451/74/DV_8_3866935_Stimmung_01_20181029101747.jpg
https://cdn.hornbach.de/data/shop/D04/001/703/124/182/DV_8_7137675_Querschnitt_Illu_01_20190605174758.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/185/561/DV_8_3556589_06_4c_DE_20190605174758.jpg
Product 28
https://cdn.hornbach.de/data/shop/D04/001/700/370/64/DV_8_3557560b_20181025164756.jpg
https://cdn.hornbach.de/data/shop/D04/001/700/451/74/DV_8_3866935_Stimmung_01_20181029101747.jpg
https://cdn.hornbach.de/data/shop/D04/001/703/124/182/DV_8_7137675_Querschnitt_Illu_01_20190605174758.jpg
Product 29
https://cdn.hornbach.de/data/shop/D04/001/700/370/64/DV_8_3557560b_20181025164756.jpg
https://cdn.hornbach.de/data/shop/D04/001/700/451/74/DV_8_3866935_Stimmung_01_20181029101747.jpg
https://cdn.hornbach.de/data/shop/D04/001/703/124/182/DV_8_7137675_Querschnitt_Illu_01_20190605174758.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/185/561/DV_8_3556589_06_4c_DE_20190605174758.jpg

https://cdn.hornbach.de/data/shop/D04/001/700/370/74/DV_8_3557764_02_4c_DE_20190416114651.jpg
https://cdn.hornbach.de/data/shop/D04/001/704/545/59/DV_8_5115937_01_4c_DE_20200226131753.jpg
https://cdn.hornbach.de/data/shop/D04/001/700/370/75/DV_8_3557764_01_20190819141755.jpg
Product 61
https://cdn.hornbach.de/data/shop/D04/001/700/370/74/DV_8_3557764_02_4c_DE_20190416114651.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/872/444/DV_8_5010859_01_4c_DE_20200921071652.jpg
https://cdn.hornbach.de/data/shop/D04/001/704/545/57/DV_8_5010847_01_4c_DE_20200916134700.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/872/464/DV_8_5010859_02_4c_DE_20200916151654.jpg
Product 62
https://cdn.hornbach.de/data/shop/D04/001/700/370/74/DV_8_3557764_02_4c_DE_20190416114651.jpg
Product 63
https://cdn.hornbach.de/data/shop/D04/001/780/491/178/58/DV_8_8121778_01_4c_DE_20200210141755.jpg
Product 64
https://cdn.hornbach.de/data/shop/D04/001/780/491/178/59/DV_8_8121780_01_4c_DE_20120802101406.jpg
Prod

https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 117
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 118
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 119
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/156/DV_8_6178703_01_4c_DE_20161018181042.jpg
Product 120
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 121
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 122
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 123
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 124
https://cdn.hornbach.de/data/shop/D04/001/780/491/008/157/DV_8_6178704_01_4c_DE_20161018182715.jpg
Product 125
h

https://cdn.hornbach.de/data/shop/D04/001/703/123/556/DV_8_3867634_Stimmung_01_20200504154758.jpg
Product 163
https://cdn.hornbach.de/data/shop/D04/001/703/134/924/DV_8_3867635_01_20190522134757.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/442/71/DV_8_3867634_02_4c_DE_20200504154758.jpg
https://cdn.hornbach.de/data/shop/D04/001/703/123/556/DV_8_3867634_Stimmung_01_20200504154758.jpg
https://cdn.hornbach.de/data/shop/D04/001/705/212/24/DV_8_7174963_05_4c_DE_20200504154758.jpg
Product 164
https://cdn.hornbach.de/data/shop/D04/001/703/134/924/DV_8_3867635_01_20190522134757.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/442/71/DV_8_3867634_02_4c_DE_20200504154758.jpg
https://cdn.hornbach.de/data/shop/D04/001/703/123/556/DV_8_3867634_Stimmung_01_20200504154758.jpg
Product 165
https://cdn.hornbach.de/data/shop/D04/001/703/134/924/DV_8_3867635_01_20190522134757.jpg
https://cdn.hornbach.de/data/shop/D04/001/780/491/442/71/DV_8_3867634_02_4c_DE_20200504154758.jpg
https://cdn.hor

In [2499]:
images_df = pd.concat([images_df, images_df_missed])   

### Keywords

In [2538]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [2539]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [2540]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

### Updating Product types

In [2629]:
df_proc['Product Type Comb'] = np.nan
df_proc['Product Type Comb'].loc[0] = ''

In [2630]:
for i in df_proc.index:
    df_proc['Product Type Comb'].loc[i] = []
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in df_proc['Combined'].loc[i].lower():
            if any([key_word.lower() in x.lower() for x in df_proc['Product Type Comb'].loc[i]]):
                continue
            
            else:
                df_proc['Product Type Comb'].loc[i].append(key_word)
    df_proc['Product Type Comb'].loc[i] = ', '.join(df_proc['Product Type Comb'].loc[i])

In [2631]:
df_proc['Product Type Comb'] = df_proc['Product Type Comb'].apply(lambda x: np.nan if x=='' else x)

In [2632]:
wrong_products = ['Befestigungssysteme', 'Zubehör']

In [2633]:
df_proc = df_proc[~df_proc['Product Type Comb'].isin(wrong_products)]
df_proc.shape

(3446, 192)

In [2634]:
df_proc['Product Type'] = df_proc['Artikeltyp'].fillna(df_proc['Product Type Comb'])

In [2635]:
df_proc['Product Type'].value_counts()

Leiste                    1738
Kantholz                   900
Platte                     400
Profilholz                  85
Ausbauplatte                65
Paneel                      62
Diele                       52
Brett                       37
Latte                       28
Kreuzrahmen                 17
Kante                       11
Kantholz, Kreuzrahmen       10
Box                         10
Massivholzdiele              8
Mehrzweckplatte              6
Bohle                        5
Unterkonstruktion            5
Diele, Massivholzdiele       2
Laufschienenset              2
Kunststoffplatte             2
Bausatz                      1
Name: Product Type, dtype: int64

## Checking Products

In [2638]:
df_proc[df_proc['Product Type']==''].shape#['Artikeltyp'].value_counts()#.nunique()

(0, 193)

### Matching keywords

In [2639]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2640]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined', 'Surface Quality']

In [2641]:
df_proc['Matched Keywords'] = np.nan
df_proc['Other Keywords'] = np.nan
df_proc = df_proc.fillna('-777')
keywords = keywords.fillna('-999')

In [2642]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
df_proc = df_proc.apply(get_keywords, axis=1)

CPU times: user 1min 5s, sys: 1.18 s, total: 1min 6s
Wall time: 1min 12s


In [2643]:
df_proc[['Matched Keywords','Other Keywords']].isna().sum()/df_proc.shape[0]*100

Matched Keywords    5.020313
Other Keywords      5.020313
dtype: float64

### Cleaning NaNs



In [2644]:
for col in df_proc.columns:
    df_proc[col] =  df_proc[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [2645]:
df_proc[(df_proc['Matched Keywords'].isna())].shape

(173, 195)

In [2646]:
df_proc[(df_proc['Matched Keywords'].isna()) & 
          (df_proc['Wood Type']=='Holz')].shape

(1, 195)

In [2647]:
df_proc[(df_proc['Matched Keywords'].isna()) #&
          ]['Product Type'].value_counts()#.unique()#

Leiste             132
Paneel              38
Laufschienenset      2
Kante                1
Name: Product Type, dtype: int64

In [2648]:
df_proc[df_proc['SKU'].str.contains('8099459')].shape

(1, 195)

### Pre-cleaning

In [2649]:
for col in df_proc.columns:
    df_proc[col] = df_proc[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [2650]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1LVxutY3DilXH6t3P0dI-oYl3bA9nWcmwUva7CukV98c'

In [2651]:
sheets_clean = df_proc.copy()
sheets_clean.shape

(3446, 195)

In [2653]:
sheets_clean['Website Name'] = 'www.hornbach.de'

In [2654]:
images_df.drop_duplicates('URL').shape

(2290, 2)

In [2655]:
sheets_clean=pd.merge(sheets_clean, images_df.drop_duplicates('URL'), on='URL', how='left')

sheets_clean.shape[0], sheets_clean['Images'].isna().sum()

(3446, 0)

In [2657]:
ren_cols = {'Title':'Product Title',
            'Breite':'Width (mm)',
            'Länge':'Length (mm)', 
            'Stärke':'Thickness (mm)',
            'Price':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Quality', 'Surface Treatment','Drying Method', 'Surface Quality',
        'Sorting Class','SKU','Images']

sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [2658]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]
sheets_clean.shape

(3446, 16)

In [2659]:
sheets_clean = sheets_clean.drop_duplicates(['URL','SKU'])
sheets_clean.shape

(2238, 16)

In [2661]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

0.0

In [2662]:
d2g.upload(sheets_clean[cols].fillna('-'),
           spreadsheet_key,
           'Hornbach',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'Hornbach' id:1932360247>